In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230306'
and yyyymmdd <= '20230310'
and city_name in ('Kolkata')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,640581ef1d8aa83196bd7828,1.468,883cf2c645fffff,22.574360,88.354668,Kolkata,883cf2d4a1fffff,22.593809,883cf2d4a1fffff,22.593865,88.338472,883cf2c645fffff,22.574360,88.354668,88.338440,113223,11,1130,Link,11:59,1,20230306,dropped,None
1,640581f29479fa4c8cfc18f7,1.204,883cf2c627fffff,22.539429,88.376526,Kolkata,883cf2c62dfffff,22.545424,883cf2c62dfffff,22.545554,88.363830,883cf2c627fffff,22.539429,88.376526,88.363541,113226,11,1130,Link,11:59,1,20230306,dropped,None
2,6405827f3fc4ae02c371aa1f,2.092,883cf2c703fffff,22.514034,88.347145,Kolkata,883cf2c555fffff,22.476023,883cf2c555fffff,22.476016,88.367714,883cf2c703fffff,22.514034,88.347145,88.367714,113447,11,1130,Link,11:59,1,20230306,dropped,None
3,64058286275d6f525c953583,NaN,883cf2d483fffff,22.618517,88.353280,Kolkata,883cf2d481fffff,22.620996,883cf2d483fffff,22.621422,88.349593,883cf2d59bfffff,22.580940,88.342326,88.348806,113454,11,1130,Link,11:59,1,20230306,expired,None
4,640582b8af33c74eaa8ba5a5,0.569,883cf2c6d5fffff,22.580664,88.444656,Kolkata,883cf2c699fffff,22.582376,883cf2c699fffff,22.582396,88.460064,883cf2c6d5fffff,22.580664,88.444656,88.459633,113544,11,1130,Link,11:59,1,20230306,dropped,None


In [5]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,640581ef1d8aa83196bd7828,1.468,883cf2c645fffff,22.574360,88.354668,Kolkata,883cf2d4a1fffff,22.593809,883cf2d4a1fffff,22.593865,88.338472,883cf2c645fffff,22.574360,88.354668,88.338440,113223,11,1130,Link,11:59,1,20230306,dropped,None,23,32,06,1
1,640581f29479fa4c8cfc18f7,1.204,883cf2c627fffff,22.539429,88.376526,Kolkata,883cf2c62dfffff,22.545424,883cf2c62dfffff,22.545554,88.363830,883cf2c627fffff,22.539429,88.376526,88.363541,113226,11,1130,Link,11:59,1,20230306,dropped,None,26,32,06,1
2,6405827f3fc4ae02c371aa1f,2.092,883cf2c703fffff,22.514034,88.347145,Kolkata,883cf2c555fffff,22.476023,883cf2c555fffff,22.476016,88.367714,883cf2c703fffff,22.514034,88.347145,88.367714,113447,11,1130,Link,11:59,1,20230306,dropped,None,47,34,06,1
3,64058286275d6f525c953583,NaN,883cf2d483fffff,22.618517,88.353280,Kolkata,883cf2d481fffff,22.620996,883cf2d483fffff,22.621422,88.349593,883cf2d59bfffff,22.580940,88.342326,88.348806,113454,11,1130,Link,11:59,1,20230306,expired,None,54,34,06,1
4,640582b8af33c74eaa8ba5a5,0.569,883cf2c6d5fffff,22.580664,88.444656,Kolkata,883cf2c699fffff,22.582376,883cf2c699fffff,22.582396,88.460064,883cf2c6d5fffff,22.580664,88.444656,88.459633,113544,11,1130,Link,11:59,1,20230306,dropped,None,44,35,06,1


In [6]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Kolkata')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [7]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,640581ef1d8aa83196bd7828,1.468,883cf2c645fffff,22.574360,88.354668,Kolkata,883cf2d4a1fffff,22.593809,883cf2d4a1fffff,22.593865,88.338472,883cf2c645fffff,22.574360,88.354668,88.338440,113223,11,1130,Link,11:59,1,20230306,dropped,None,23,32,06,1,Salkia H,Sonapatti
1,640581f29479fa4c8cfc18f7,1.204,883cf2c627fffff,22.539429,88.376526,Kolkata,883cf2c62dfffff,22.545424,883cf2c62dfffff,22.545554,88.363830,883cf2c627fffff,22.539429,88.376526,88.363541,113226,11,1130,Link,11:59,1,20230306,dropped,None,26,32,06,1,Parkstreet,Park Circus
2,6405827f3fc4ae02c371aa1f,2.092,883cf2c703fffff,22.514034,88.347145,Kolkata,883cf2c555fffff,22.476023,883cf2c555fffff,22.476016,88.367714,883cf2c703fffff,22.514034,88.347145,88.367714,113447,11,1130,Link,11:59,1,20230306,dropped,None,47,34,06,1,NetajiNagar,Kalighat
3,64058286275d6f525c953583,NaN,883cf2d483fffff,22.618517,88.353280,Kolkata,883cf2d481fffff,22.620996,883cf2d483fffff,22.621422,88.349593,883cf2d59bfffff,22.580940,88.342326,88.348806,113454,11,1130,Link,11:59,1,20230306,expired,None,54,34,06,1,Don Bosco Lake H,Howrah Station
4,640582b8af33c74eaa8ba5a5,0.569,883cf2c6d5fffff,22.580664,88.444656,Kolkata,883cf2c699fffff,22.582376,883cf2c699fffff,22.582396,88.460064,883cf2c6d5fffff,22.580664,88.444656,88.459633,113544,11,1130,Link,11:59,1,20230306,dropped,None,44,35,06,1,NewTown Action Area 1B,Saltlake sector 5


In [8]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,883cf21105fffff,3
1,20230306,17,883cf21167fffff,1
2,20230306,17,883cf21301fffff,1
3,20230306,17,883cf21303fffff,5
4,20230306,17,883cf2130bfffff,6


In [18]:
cluster_grouped_morning = df[df['hour'].isin(['08', '09', '10', '11'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_morning = cluster_grouped_morning.sort_values(by = 'order_id', ascending=False)
cluster_grouped_evening = cluster_grouped_evening.sort_values(by = 'order_id', ascending=False)

top_clusters_morning = cluster_grouped_morning.head(40)
top_clusters_morning['orders_per_day'] = top_clusters_morning['order_id'].apply(lambda x: x/5)
top_clusters_morning['orders_per_hour'] = top_clusters_morning['orders_per_day'].apply(lambda x: x/4)
top_clusters_morning['orders_per_20_sec_batch'] = top_clusters_morning['orders_per_hour'].apply(lambda x: x/180)

top_clusters_evening = cluster_grouped_evening.head(40)
top_clusters_evening['orders_per_day'] = top_clusters_evening['order_id'].apply(lambda x: x/5)
top_clusters_evening['orders_per_hour'] = top_clusters_evening['orders_per_day'].apply(lambda x: x/4)
top_clusters_evening['orders_per_20_sec_batch'] = top_clusters_evening['orders_per_hour'].apply(lambda x: x/180)
top_clusters_evening

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
338,Taltala,2776,555.2,138.80,0.771111
301,Saltlake sector 5,2747,549.4,137.35,0.763056
271,Parkstreet,2158,431.6,107.90,0.599444
33,Ballygunge,1961,392.2,98.05,0.544722
153,Jadavpur,1936,387.2,96.80,0.537778
339,Tangra,1648,329.6,82.40,0.457778
321,Sonapatti,1578,315.6,78.90,0.438333
312,Sector IV,1228,245.6,61.40,0.341111
311,Sector 1,1220,244.0,61.00,0.338889
164,Kalighat,1128,225.6,56.40,0.313333


In [19]:
top_clusters_morning

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
152,Jadavpur,1361,272.2,68.05,0.378056
336,Taltala,1260,252.0,63.00,0.350000
337,Tangra,1103,220.6,55.15,0.306389
149,Howrah Station,1037,207.4,51.85,0.288056
170,Kasba,958,191.6,47.90,0.266111
324,South Dum Dum,896,179.2,44.80,0.248889
237,NetajiNagar,861,172.2,43.05,0.239167
24,Bagmari,824,164.8,41.20,0.228889
31,Ballygunge,806,161.2,40.30,0.223889
26,Baguiati,804,160.8,40.20,0.223333


In [9]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [10]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [11]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '5c124a1e0241913903a42fe1'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [12]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,cold,Alipore,Monday,1200,Monday-1200,0,0.0,0,0.0,0.0,0.0,0.0,0.206305,0.118455,0.396154,0.096419,0.204333,20221220,HOT_COLD_3_0,5c124a1e0241913903a42fe1
1,cold,Bagberia,Friday,0900,Friday-0900,0,0.0,0,0.0,0.0,0.0,0.0,0.324259,0.324259,0.000000,0.000000,0.162130,20221220,HOT_COLD_3_0,5c124a1e0241913903a42fe1
2,cold,Baksara H,Monday,1500,Monday-1500,0,0.0,0,0.0,0.0,0.0,0.0,0.051772,0.437575,0.047692,0.095385,0.158106,20221220,HOT_COLD_3_0,5c124a1e0241913903a42fe1
3,medium,Baksara H,Monday,1700,Monday-1700,0,0.0,0,0.0,0.0,0.0,0.0,0.268322,0.546358,0.217308,0.273077,0.326266,20221220,HOT_COLD_3_0,5c124a1e0241913903a42fe1
4,medium,Baksara H,Monday,1800,Monday-1800,0,0.0,0,0.0,0.0,0.0,0.0,0.272236,0.547103,0.266667,0.314103,0.350027,20221220,HOT_COLD_3_0,5c124a1e0241913903a42fe1


In [13]:
df_mismatch = df_mismatch[df_mismatch['run_date'] == df_mismatch['run_date'].max()]

In [14]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
9701,super-cold,Baksara H,Tuesday,1900,Tuesday-1900,0,0.0,0,0.0,0.0,0.0,0.0,0.057931,0.250591,0.074359,0.131410,0.128573,20230314,HOT_COLD_3_0,5c124a1e0241913903a42fe1,19
9957,super-cold,Baksara H,Tuesday,1800,Tuesday-1800,0,0.0,0,0.0,0.0,0.0,0.0,0.068211,0.240846,0.124722,0.134722,0.142125,20230314,HOT_COLD_3_0,5c124a1e0241913903a42fe1,18
9958,cold,Baksara H,Tuesday,2100,Tuesday-2100,0,0.0,0,0.0,0.0,0.0,0.0,0.169805,0.432996,0.149038,0.245833,0.249418,20230314,HOT_COLD_3_0,5c124a1e0241913903a42fe1,21
9959,medium,Baksara H,Tuesday,1500,Tuesday-1500,0,0.0,0,0.0,0.0,0.0,0.0,0.259424,0.477994,0.278889,0.382500,0.349702,20230314,HOT_COLD_3_0,5c124a1e0241913903a42fe1,15
9960,medium,Baksara H,Tuesday,1400,Tuesday-1400,0,0.0,0,0.0,0.0,0.0,0.0,0.226154,0.340333,0.293403,0.410069,0.317490,20230314,HOT_COLD_3_0,5c124a1e0241913903a42fe1,14


In [15]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,6406141660804939c421fdce,0.832,883cf2c46bfffff,22.516653,88.381935,Kolkata,883cf2c44dfffff,22.528196,883cf2c44dfffff,22.528196,88.379211,883cf2c46bfffff,22.516653,88.381935,88.379211,215558,21,2145,Link,21:59,1,20230306,dropped,None,58,55,06,1,Kasba,Molla Parra,9,Monday,0.175160
1,6406145160804939c421fe2a,NaN,883cf28943fffff,22.634710,88.398102,Kolkata,883cf28943fffff,22.635174,883cf28943fffff,22.634597,88.397885,883cf28945fffff,22.624915,88.387755,88.398186,215657,21,2145,Link,21:59,1,20230306,customerCancelled,order cancelled before rider accepted,57,56,06,1,Bediapara,Sinthee,1,Monday,0.230268
2,6406075bc1cab2572ae6dfc0,0.803,883cf2d4a1fffff,22.595432,88.339729,Kolkata,883cf2c74bfffff,22.541960,883cf2c74bfffff,22.541947,88.326728,883cf2d4a1fffff,22.595432,88.339729,88.326714,210139,21,2100,Link,21:00,1,20230306,dropped,None,39,01,06,1,Khidirpur,Salkia H,30,Monday,0.208618
3,640607bf60804939c421ec92,NaN,883cf28931fffff,22.604761,88.425636,Kolkata,883cf28931fffff,22.604774,883cf28931fffff,22.604733,88.425674,883cf2c6ddfffff,22.590064,88.433320,88.425591,210319,21,2100,Link,21:00,1,20230306,customerCancelled,order cancelled before rider accepted,19,03,06,1,Kestopur,Krishnapur,13,Monday,0.337193
4,640608f81d8aa83196be5711,0.387,883cf2c753fffff,22.537285,88.354301,Kolkata,883cf2c753fffff,22.537279,883cf2c753fffff,22.537259,88.354324,883cf2c54bfffff,22.485670,88.355435,88.354317,210832,21,2100,Link,21:00,1,20230306,customerCancelled,Change of plans,32,08,06,1,Ballygunge,NetajiNagar,32,Monday,0.474831


In [16]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]

In [17]:
df_evening_filter.to_csv('kolkata_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('kolkata_demand_supply_morning_vw2.csv', index = False)

In [18]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [19]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,17,1338
1,20230227,18,1595
2,20230227,19,1795
3,20230227,20,1689
4,20230227,21,1399
5,20230228,17,1257
6,20230228,18,1519
7,20230228,19,1787
8,20230228,20,1624
9,20230228,21,1371


In [20]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [21]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
169,20230302,18,Thousand Lights Metro,92,43,2.1395
119,20230301,18,Thousand Lights Metro,87,50,1.7400
164,20230302,18,Nungambakkam,392,235,1.6681
100,20230301,17,Alwarpet,71,44,1.6136
110,20230301,18,Alwarpet,118,76,1.5526


In [27]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean()

hour
17    0.918695
18    1.022725
19    0.714440
20    0.515355
21    0.482165
Name: demand_to_supply_ratio, dtype: float64

In [22]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [23]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [24]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [25]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [26]:
df_evening_filter.to_csv('chennai_demand_supply_evening.csv', index = False)